In [1]:
from sim_utils import plot_utils, run_sim
from graph_create import ring_lattice
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

In [2]:
# Constants
num_vertices = 100
regularity = 4

In [3]:
def create_rl(n, k, p, r):
    return ring_lattice(k, n, p, r)

## Evenly distributed resources

In [4]:
run_sim(num_vertices, regularity, create_rl)

## Powerlaw distributed resources

In [5]:
run_sim(num_vertices, regularity, create_rl, True)